<a href="https://colab.research.google.com/github/zhiyuan-95/Amex-default-prediction-/blob/main/combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=t3sIXw1LRkUHLlHka70VUEWNqdgLaf&prompt=consent&access_type=offline&code_challenge=xtmWTy6bc5Ab5n1153R3pp8vX816Q4JaFhbtI1418TQ&code_challenge_method=S256

Enter authorization code: 4/0AZEOvhWiWI5vvMmLEgZkFBpQ3YsSXUn3vGPa3KZizmBG7aJ5il6HWV8QOuFnqGM_M9uc1g

You are now logged in as [zhiyuan.jin1201@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID

In [ ]:
!gcloud dataproc clusters create combine --enable-component-gateway --region us-west1 --zone us-west1-a --master-machine-type n1-standard-4 --master-boot-disk-size 500 --num-workers 2 --worker-machine-type n1-standard-4 --worker-boot-disk-size 100 --image-version 2.0-debian10 --project bda-12345

Waiting on operation [projects/bda-12345/regions/us-west1/operations/820e090b-07bf-3c09-8387-e81f1ff3c8af].

Created [https://dataproc.googleapis.com/v1/projects/bda-12345/regions/us-west1/clusters/combine] Cluster placed in zone [us-west1-a].


In [ ]:
!gcloud config set project bda-12345
!gcloud config set compute/region us-west1
!gcloud config set compute/zone us-west1-a
!gcloud config set dataproc/region us-west1

Updated property [core/project].
Updated property [compute/region].
Updated property [compute/zone].
Updated property [dataproc/region].


In [ ]:
%%writefile combine_test.py
import itertools
import pyspark
from pyspark import SparkContext
sc = SparkContext()
test = sc.textFile('gs://aedp/expanded_data/expanded_test/part-*')\
          .coalesce(1)\
          .saveAsTextFile('gs://aedp/expanded_data/test')


Overwriting combine_test.py


In [ ]:
!gcloud dataproc jobs submit pyspark --cluster combine combine_test.py

Job [35a0231619ac4684b6fdcd6f04e18516] submitted.
Waiting for job output...
23/07/29 20:10:12 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/07/29 20:10:12 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/07/29 20:10:12 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/07/29 20:10:12 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
23/07/29 20:10:12 INFO org.sparkproject.jetty.util.log: Logging initialized @4465ms to org.sparkproject.jetty.util.log.Slf4jLog
23/07/29 20:10:12 INFO org.sparkproject.jetty.server.Server: jetty-9.4.40.v20210413; built: 2021-04-13T20:42:42.668Z; git: b881a572662e1943a14ae12e7e1207989f218b74; jvm 1.8.0_372-b07
23/07/29 20:10:12 INFO org.sparkproject.jetty.server.Server: Started @4571ms
23/07/29 20:10:12 INFO org.sparkproject.jetty.server.AbstractConnector: Started ServerConnector@29a64e0c{HTTP/1.1, (http/1.1)}{0.0.0.0:43847}
23/07/29 20:10:13 INFO org.apache.hadoop.yarn.client.RMPro

In [ ]:
%%writefile train.py
import itertools
import pyspark
from pyspark import SparkContext
sc = SparkContext()
train = sc.textFile('gs://aedp/expanded_data/expanded_data/part-*')\
          .coalesce(1)\
          .saveAsTextFile('gs://aedp/expanded_data/train')


Overwriting train.py


In [ ]:
!gcloud dataproc jobs submit pyspark --cluster combine train.py

Job [c2f57f9433d64db3ad4b5caf2ffed84b] submitted.
Waiting for job output...
23/07/24 01:32:07 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/07/24 01:32:07 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/07/24 01:32:07 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/07/24 01:32:07 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
23/07/24 01:32:08 INFO org.sparkproject.jetty.util.log: Logging initialized @3631ms to org.sparkproject.jetty.util.log.Slf4jLog
23/07/24 01:32:08 INFO org.sparkproject.jetty.server.Server: jetty-9.4.40.v20210413; built: 2021-04-13T20:42:42.668Z; git: b881a572662e1943a14ae12e7e1207989f218b74; jvm 1.8.0_372-b07
23/07/24 01:32:08 INFO org.sparkproject.jetty.server.Server: Started @3761ms
23/07/24 01:32:08 INFO org.sparkproject.jetty.server.AbstractConnector: Started ServerConnector@6a12c15a{HTTP/1.1, (http/1.1)}{0.0.0.0:45481}
23/07/24 01:32:08 INFO org.apache.hadoop.yarn.client.RMPro

In [ ]:
!gcloud dataproc clusters delete split

The cluster 'split' and all attached disks will be deleted.

Do you want to continue (Y/n)?  y

Waiting on operation [projects/bda-12345/regions/us-west1/operations/b72fdeb1-65ae-3830-944f-b08dfcd5031f].
Deleted [https://dataproc.googleapis.com/v1/projects/bda-12345/regions/us-west1/clusters/split].
